In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign, expand
from IPython.display import display, HTML, Math

In [2]:
def large(expr):
  latex_code = f"\\Large {{ {latex(expr)} }}"
  display(Math(latex_code))

In [3]:
delta_b_negative = False
discrete = True
simp = True

In [4]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'l', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, l_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, l_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [29]:
deltaByDelta_b = Le(Delta_b, floor(Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))) # upper bound
deltaByDelta_s = Ge(Delta_s, ceiling(-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))) # lower bound
display(deltaByDelta_b)
display(deltaByDelta_s)

Delta_b <= floor(Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))

Delta_s >= ceiling(-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))

In [28]:
# this seems to be derived from spotByDelta
def deltaBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = (s - l * w) / w # at this point, this value is negative for buying and positive for selling
  # rounding towards zero in all cases, as we are interested in the maximum delta-capacity of a given spot price
  if asset == 'b':
    if delta_b_negative:
      if discrete:
        f = ceiling(f)
    else:
      if discrete:
        f = floor(-f)
      else:
        f = -f

  else:
    if discrete:
      f = floor(f)

  if simp:
    f = simplify(f)
  return Le(Delta, f)

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
display( deltaBySpot['b'])
display(deltaBySpot['s'])

Delta_b <= l_b + floor(-s_b/w_b)

Delta_s <= -l_s + floor(s_s/w_s)

In [7]:
def spotByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  if (not delta_b_negative) and asset == 'b':
    f = w * (l - Delta)
  else:
    f = w * (l + Delta)
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(s_b, spotByDelta['b']))
display(Eq(s_s, spotByDelta['s']))

Eq(s_b, w_b*(-Delta_b + l_b))

Eq(s_s, w_s*(Delta_s + l_s))

In [34]:
# upper bound because deltaByDelta_b and deltaBySpot['s'] both are upper bounds
spotBySpot_s = Le(s_b, spotByDelta['b'].subs({Delta_b: deltaByDelta_b.rhs, Delta_s: deltaBySpot['s'].rhs}).simplify())
spotBySpot_s

s_b <= w_b*(l_b - floor(l_b*w_b*(-l_s + floor(s_s/w_s))/(-l_s*w_b + w_b*floor(s_s/w_s) + w_s*floor(s_s/w_s))))

should not attempt the same for s_s, as deltaByDelta_s is a lower bound and deltaBySpot['b] an upper bound

In [43]:
def spotByExp_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if discrete:
    f = floor(f) # always rounding down because of how spot is computed from exp
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(asset, *all_symbols[asset]) for asset in assets}
display(spotByExp['b'])
display(spotByExp['s'])

floor(a_b*((j_b + 1)/j_b)**e_b)

floor(a_s*((j_s + 1)/j_s)**e_s)

In [47]:
def expBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = log(s / a) / log(1 + 1/j)
  if discrete:
    f = ceiling(f) # always rounding up because of how spot is computed from exp
  if simp:
    f = simplify(f)
  return Le(e, f)
expBySpot = {asset: expBySpot_(asset, *all_symbols[asset]) for asset in assets}
display(expBySpot['b'])
display(expBySpot['s'])

e_b <= ceiling(log((s_b/a_b)**(1/log((j_b + 1)/j_b))))

e_s <= ceiling(log((s_s/a_s)**(1/log((j_s + 1)/j_s))))

In [48]:
spotBySpot_s.subs({s_s: spotByExp['s']}).simplify()

s_b <= w_b*(l_b - floor(l_b*w_b*(-l_s + floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))/(-l_s*w_b + w_b*floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s) + w_s*floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))))

In [50]:
expBySpot['b'].subs({s_b: spotBySpot_s.rhs.subs({s_s: spotByExp['s']}).simplify()}).simplify()

e_b <= ceiling(log((w_b*(l_b - floor(l_b*w_b*(-l_s + floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))/(-l_s*w_b + w_b*floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s) + w_s*floor(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))))/a_b)**(1/log((j_b + 1)/j_b))))